In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import seaborn as sns; sns.set()
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
import re
from bs4 import BeautifulSoup
from urllib import request
from urllib.request import urlopen
import time
import pickle
from sklearn.preprocessing import StandardScaler

In [3]:
predict_day = input('予測したい日をyyyymmdd型で入力:')
jcd = input('予測したい場を2桁で入力:')
rno = input('予測したいラウンドを入力:')

In [4]:
url='https://www.boatrace.jp/owpc/pc/race/racelist?rno='+str(rno)+\
    '&jcd='+str(jcd).zfill(2)+'&hd='+str(predict_day)
time.sleep(1)
response=request.urlopen(url)
soup=BeautifulSoup(response,'html.parser')
response.close()
data = []
for i,j in zip(range(0,6),range(0,11,2)):
    table_class = soup.find_all('div',class_='is-fs11')[j].get_text()
    table_class = table_class.replace(' ','').replace('\n',',').replace('/','')
    table_class = table_class.split(',')
    race_class = table_class[1]
    table = soup.find_all('tbody',class_='is-fs12')[i].get_text()
    # 番号を抽出する
    number = re.search(r'\d+', table).group()
    # 名前を抽出する
    name = re.search(r'\n\n(.+)\n\n(.+)\n', table).group(1).strip().replace('\u3000','')
    # 場所を抽出する
    location = re.search(r'\n\n(.+)\n\n(.+)\n', table).group(2).strip().replace('/','')
    # 年齢と体重を抽出する
    age, weight = re.search(r'(\d+歳)/(\d+\.\d+kg)', table).groups()
    # 数字のリストを抽出する
    numbers = re.findall(r'(\d+\.\d+)', table)
    a_numbers = re.findall(r'(\d+)',table)
    location = location[:2]
    age = age.replace('歳','')
    weight = weight.replace('kg','')
    motor = a_numbers[22]
    boat= a_numbers[27]
    
    data.append([number,name,location,age,*numbers,motor,boat,race_class])
df_1 = pd.DataFrame(data,columns=['position','name','area','age','weight','st_time_ave','win',\
        'win_p_2','win_p','a_win','a_win_p_2','a_win_p','motor_p_2','motor_p','boat_p_2','boat_p','motor','boat','class'])
url = 'https://www.boatrace.jp/owpc/pc/race/beforeinfo?rno='+str(rno)+\
        '&jcd='+str(jcd).zfill(2)+'&hd='+str(predict_day)
time.sleep(1)
response = request.urlopen(url)
soup = BeautifulSoup(response,'html.parser')
response.close()
data_2 = []
for i in range(0,6):
    table = soup.find_all('tbody',class_='is-fs12')[i].get_text()
    table = table.replace('\n\n','').replace('\u3000','').replace('\n',',')
    table_data = table.split(',')
    position_name = table_data[0]
    name = position_name[1:]
    display = table_data[2]
    table2 = soup.find_all('tbody',class_='is-fs12')[i].get_text()
    table2 = table2.replace('\u3000','').replace('\n\n','').replace('\n',',').split(',')
    st_times = table2[2]
    table3 = soup.find('div',class_='weather1_body').get_text()
    table3 = table3.replace('\n\n\n\n','').replace('\n',',')
    table3_data = table3.split(',')
    weather = table3_data[3]
    wind_strong = table3_data[5].replace('m水温','')
    wave = table3_data[8].replace('cm','')
    data_2.append([name,display,st_times,weather,wind_strong,wave])
df_2 = pd.DataFrame(data_2,columns=['name','display','st_time','weather','wind_strong','wave'])
# dfの前処理
df = pd.merge(df_1,df_2,on=['name'])
float_columns = ['age','weight','position','st_time_ave','win','win_p_2', 'win_p', 'a_win', 'a_win_p_2', 'a_win_p', 'motor_p_2',\
'motor_p', 'boat_p_2', 'boat_p', 'motor', 'boat', 'display', 'st_time',\
'wind_strong', 'wave']
for i in float_columns:
    df[i] = df[i].astype(float)
std_columns = ['win','win_p_2', 'win_p', 'a_win', 'a_win_p_2', 'a_win_p', 'motor_p_2',\
    'motor_p', 'boat_p_2', 'boat_p']
for i in std_columns:
    win_mean=df[i].mean()
    win_std=df[i].std()
    if win_std==0: #もしdf_stdが同じ値だとwin_stdが0になってしまう
        df[i]=50.0
    else:
        df[i] = df[i].apply(lambda x : ((x - win_mean)*10/win_std+50))
# int型に変換
int_columns = ['age','position']
for i in int_columns:
    df[i] = df[i].astype(int)
def encode(df, col):
    ###この方法だと場合によって最大値が変化するデータでは正確な値は出ない 例：月の日数が30日や31日の場合がある
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
    return df
df['race_date'] = predict_day
df['race_date'] = pd.to_datetime(df['race_date'])
df['day_of_year']= df['race_date'].apply(lambda x: x.timetuple().tm_yday)
encode(df,'day_of_year')
dict_stadium = {'01':'桐生','02':'戸田','03':'江戸川','04':'平和島',\
                            '05':'多摩川','06':'浜名湖','07':'蒲郡','08':'常滑',\
                            '09':'津','10':'三国','11':'びわこ','12':'住之江',\
                            '13':'尼崎','14':'鳴門','15':'丸亀','16':'児島',\
                            '17':'宮島','18':'徳山','19':'下関','20':'若松',\
                            '21':'芦屋','22':'福岡','23':'唐津','24':'大村'}
df['place'] = dict_stadium[jcd]

In [5]:
drop_columns = ['race_date']
df.drop(drop_columns,axis=1,inplace=True)

In [6]:
categorical_features = ['place','name','area','weather','class']
for list in categorical_features:
    df[list] = df[list].astype('category')

In [7]:
df = pd.get_dummies(df,columns=categorical_features,drop_first=True)

In [8]:
with open('model_gbm_2.pkl','rb') as model_file:
    model = pickle.load(model_file)

In [9]:
y_pred_prob = model.predict(df,predict_disable_shape_check=True)

In [10]:
y_pred = np.argmax(y_pred_prob,axis=1)

In [11]:
y_pred

array([0, 0, 0, 0, 0, 0])

In [12]:
df_pred_prob = pd.DataFrame({'四着以下':y_pred_prob[:,0], '一着':y_pred_prob[:,1], '二着':y_pred_prob[:,2],'三着':y_pred_prob[:,3]})

In [13]:
df_pred_prob

,四着以下,一着,二着,三着
0,0.363764,0.107682,0.221561,0.306993
1,0.415177,0.107190,0.174840,0.302794
2,0.370561,0.095867,0.204433,0.329140
3,0.402818,0.079911,0.188467,0.328804
4,0.361892,0.112301,0.183265,0.342542
5,0.483881,0.075265,0.154719,0.286136
